In [ ]:
# Install/update required packages to resolve TqdmWarning
import subprocess
import sys

# Upgrade jupyter and ipywidgets to fix TqdmWarning
#subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--upgrade", "jupyter", "ipywidgets"])

# Import required libraries
import cv2 as cv
import supervision as sv
from inference import get_model
import numpy as np


In [ ]:
# Initialize Inference Client for Docker-based inference server
import os
from dotenv import load_dotenv
from inference_sdk import InferenceHTTPClient

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("ROBOFLOW_API_KEY")
if not api_key:
    raise ValueError("ROBOFLOW_API_KEY not found in environment variables. Please set it in .env file.")

# Initialize client connecting to local Docker inference server
CLIENT = InferenceHTTPClient(
    api_url="http://localhost:9001",  # Local Docker inference server
    api_key=api_key
)

# Load the paraglider detection model as default
CLIENT.load_model(model_id="paraglider_recognition/8", set_as_default=True)



In [ ]:
# Load the paraglider detection model
import os
from dotenv import load_dotenv

load_dotenv()

# Get API key from environment variable
api_key = os.getenv("ROBOFLOW_API_KEY")
if not api_key:
    raise ValueError("ROBOFLOW_API_KEY not found in environment variables. Please set it in .env file.")

# Load the model using inference SDK
model = get_model(
    model_id="paraglider_recognition/8",
    api_key=api_key
)


In [ ]:
# Run inference on test image
image_path = "paraglider_recognition-8/test/images/P_20231121_114926_jpg.rf.193318024b03b5a48f7ca5bb7c37ae7d.jpg"

# Load image using OpenCV
image = cv.imread(image_path)

# Run inference on the image
results = model.infer(image)[0]

# Convert results to Supervision Detections format
detections = sv.Detections.from_inference(results)

# Create annotators for visualization
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Annotate image with bounding boxes
annotated_image = bounding_box_annotator.annotate(
    scene=image.copy(), 
    detections=detections
)

# Annotate image with labels and confidence scores
annotated_image = label_annotator.annotate(
    scene=annotated_image, 
    detections=detections, 
    labels=[f"{result.class_name} {result.confidence:.2f}" for result in detections]
)

# Display the annotated image
sv.plot_image(annotated_image, size=(8, 8))
